In [111]:
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pycocotools.coco import COCO
import pandas as pd
from tqdm import tqdm
import shutil

In [52]:
# Initialize COCO API for instance annotations
data_source = COCO('annotations.json')

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [ ]:
# Get the category names and id's from the annotations
cat_ids = data_source.getCatIds()  # Retrieve category IDs
cat_names = [cat['name'] for cat in data_source.loadCats(cat_ids)]  # Extract category names


# Define lists of items for trash and recycling
trash_list = ['Battery', 'Broken glass', 'Other plastic', 'Food waste', 'Unlabeled litter', 'Cigarette', 'Shoe', 'Styrofoam piece']
recycle_list = ['Aluminium foil', 'Aluminium blister pack', 'Carded blister pack', 'Other plastic bottle', 'Clear plastic bottle', 'Glass bottle', 'Plastic bottle cap', 'Metal bottle cap', 'Food Can', 'Aerosol', 'Drink can', 'Toilet tube', 'Other carton', 'Egg carton', 'Drink carton', 'Corrugated carton', 'Meal carton', 'Pizza box', 'Paper cup', 'Disposable plastic cup', 'Foam cup', 'Glass cup', 'Other plastic cup', 'Glass jar', 'Plastic lid', 'Metal lid', 'Magazine paper', 'Tissues', 'Wrapping paper', 'Normal paper', 'Paper bag', 'Plastified paper bag', 'Plastic film', 'Six pack rings', 'Garbage bag', 'Other plastic wrapper', 'Single-use carrier bag', 'Polypropylene bag', 'Crisp packet', 'Spread tub', 'Tupperware', 'Disposable food container', 'Foam food container', 'Other plastic container', 'Plastic glooves', 'Plastic utensils', 'Pop tab', 'Rope & strings', 'Scrap metal', 'Squeezable tube', 'Plastic straw', 'Paper straw']


# Get all of the categories and their ids as a dictionary
cat_ids = data_source.getCatIds()  # Retrieve category IDs
cat_names = [cat['name'] for cat in data_source.loadCats(cat_ids)]  # Extract category names
cat_dict = dict(zip(cat_ids, cat_names))  # Create dictionary mapping category IDs to names
cat_dict = pd.DataFrame.from_dict(cat_dict, orient='index', columns=['category'])  # Convert dictionary to DataFrame for easier manipulation


# If the category is in the trash list, label it as trash
cat_dict['label'] = 1  # Initially label all categories as recyclable
cat_dict.loc[cat_dict['category'].isin(trash_list), 'label'] = 0  # Change label to 0 for categories in the trash list


# Get the index and label as a dictionary
c = dict(zip(cat_dict.index, cat_dict.label))  # Create dictionary mapping category indices to their corresponding labels
c

In [ ]:
# Load annotations from JSON file using COCO API
data_source = COCO('annotations.json')

# Get image IDs
img_ids = data_source.getImgIds()

# Define the base path to save label files
save_base_path = 'labels/'

# Loop through each image ID and process annotations
for index, img_id in tqdm(enumerate(img_ids), desc='change .json file to .txt file'):
    # Load image information
    img_info = data_source.loadImgs(img_id)[0]
    img_name = img_info['file_name'] 
    save_name = img_info['file_name'].replace('/', '_')
    file_name = save_name.split('.')[0]
    height = img_info['height']
    width = img_info['width']
    
    # Define the path to save the label file
    save_path = save_base_path + file_name + '.txt'
    is_exist = False
    
    # Open the label file for writing
    with open(save_path, 'w+') as fp:
        # Get annotation IDs for the current image
        annotation_id = data_source.getAnnIds(img_id)
        
        # Load annotations
        annotations = data_source.loadAnns(annotation_id)
        
        # Initialize string to store annotation lines
        lines = ''
        
        # Iterate through annotations
        for annotation in annotations:
            is_exist = True
            box = annotation['bbox']
            
            # Skip annotations with invalid bounding boxes
            if box[2] < 1 or box[3] < 1:
                continue
            
            # Convert bounding box coordinates to relative values
            box[0] = round((box[0] + box[2] / 2) / width, 6)
            box[1] = round((box[1] + box[3] / 2) / height, 6)
            box[2] = round(box[2] / width, 6)
            box[3] = round(box[3] / height, 6)
            
            # Construct annotation line with label and bounding box coordinates
            lines = lines + str(c[annotation['category_id']])
            for i in box:
                lines += ' ' + str(i)
            lines += '\n'
        
        # Write annotation lines to the label file
        fp.writelines(lines)

In [113]:
# Rename and move the images to the correct folder
import os

# Define the base path to save and read images
save_base_path = 'images/'
read_base_paths = [f'batch_{i}/' for i in range(2,16)]

# Loop through each folder
for read_base_path in read_base_paths:
# Get the list of image names
    img_names = os.listdir(read_base_path)
    
    # Loop through each image and move it to the new folder
    for img_name in img_names:
        # Define the path to read the image
        read_path = f'{read_base_path}{img_name}'
        
        # Move the image to the new folder
        shutil.move(read_path, f"{save_base_path}/{img_name}")
        
        # Rename the file (optional)
        # print(read_path)
        new_filename = read_path.split('/')
        new_filename = f'{new_filename[0]}_{new_filename[1]}'
        
        os.rename(f"{save_base_path}/{img_name}", f'images/{new_filename}')
    print(f'Finished moving images from {read_base_path} to {save_base_path}')

Finished moving images from batch_2/ to images/
Finished moving images from batch_3/ to images/
Finished moving images from batch_4/ to images/
Finished moving images from batch_5/ to images/
Finished moving images from batch_6/ to images/
Finished moving images from batch_7/ to images/
Finished moving images from batch_8/ to images/
Finished moving images from batch_9/ to images/
Finished moving images from batch_10/ to images/
Finished moving images from batch_11/ to images/
Finished moving images from batch_12/ to images/
Finished moving images from batch_13/ to images/
Finished moving images from batch_14/ to images/
Finished moving images from batch_15/ to images/
